In [ ]:
import torchvision
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from imgaug import augmenters as iaa
from torchsummary import summary
import torch
import torch.nn.functional as F
from torchvision import transforms, datasets, models
import pandas as pd
import cv2
from glob import glob
from torch.utils.data import DataLoader, Dataset

In [ ]:
train_data_dir = '/Users/macbookpro/Desktop/AI SIBERIA/Computer Vision /Chapter-4/dataset/training_set/training_set'
test_data_dir = '/Users/macbookpro/Desktop/AI SIBERIA/Computer Vision /Chapter-4/dataset/test_set/test_set'

In [ ]:
# Build a Dataset class for the data
class cats_dogs(Dataset):
    def __init__(self, folder):
        super().__init__()
        cats = glob(folder+'/cats/*.jpg')
        dogs = glob(folder+'/dogs/*.jpg')
        self.fpaths = cats + dogs
        from random import shuffle, seed; seed(10); shuffle(self.fpaths)
        self.targets = [fpath.split('/')[-1].startswith('dog') for fpath in self.fpaths] # dogs=1, cats =0
    def __len__(self): return len(self.fpaths)
    def __getitem__(self, index):
        f = self.fpaths[index]
        target = self.targets[index]
        im = (cv2.imread(f)[:,:,::-1])
        im = cv2.resize(im, (224, 224))
        return torch.tensor(im/255).permute(2,0,1).float(), torch.tensor([target]).float()

In [ ]:
data = cats_dogs(train_data_dir)
im, label = data[200]

In [ ]:
plt.imshow(im.permute(1,2,0))
print(label)

In [ ]:
# define a function for the convolution that will be used in building the architecture
def conv_layer(input_channel, number_output, kernel_size, stride=1):
    return nn.Sequential(
        nn.Conv2d(input_channel, number_output, kernel_size),
        nn.ReLU(),
        nn.BatchNorm2d(number_output),
        nn.MaxPool2d(2)
    )

In [ ]:
# build the model architecture
def get_model():
    model  = nn.Sequential(
        conv_layer(3, 64, 3),
        conv_layer(64, 512, 3),
        conv_layer(512, 512, 3),
        conv_layer(512, 512, 3),
        conv_layer(512, 512, 3),
        conv_layer(512, 512, 3),
        nn.Flatten(),
        nn.Linear(512, 1),
        nn.Sigmoid()
    )
    
    loss_fn = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    return model, loss_fn, optimizer

In [ ]:
model, loss_fn, optimizer = get_model()

# model summary
summary(model, torch.zeros(1,3,224,224))

In [ ]:
def get_data():
    train = cats_dogs(train_data_dir)
    trn_dl = DataLoader(train, batch_size=32, shuffle=True, drop_last=True)
    val = cats_dogs(test_data_dir)
    val_dl = DataLoader(val, batch_size=32, shuffle=True, drop_last=True)
    return trn_dl, val_dl

We are ignoring the last batch by setting drop_last = True. We do this because the size of the last batch  may not be like that of the rest.

In [ ]:
# define a function to train the model on a batch of data
def train_batch(x, y, model, opt, loss_fn):
    model.train()
    prediction  = model(x)
    batch_loss = loss_fn(prediction, y)
    batch_loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    return batch_loss.item()
    

In [ ]:
# define accuracy function
@torch.no_grad()
def accuracy(x, y, model):
    prediction = model(x)
    is_correct = (prediction > 0.5) == y
    return is_correct.numpy().tolist()

In [ ]:
# define Validation loss calculation function
@torch.no_grad()
def val_loss(x, y, model):
    prediction = model(x)
    val_loss = loss_fn(prediction, y)
    return val_loss.item()

In [ ]:
# fetch the required dataloaders
trn_dl, val_dl = get_data()

In [ ]:
# train the model over 5 epochs
train_losses, train_accuracies = [], []
val_losses, val_accuracies = [], []

for epoch in range(5):
    print(epoch)
    
    train_epoch_losses, train_epoch_accuracies = [], []
    val_epoch_accuracies = []
    
    for ix, batch in enumerate(iter(trn_dl)):
        x, y = batch
        batch_loss = train_batch(x, y, model, optimizer, loss_fn)
        train_epoch_losses.append(batch_loss)
    train_epoch_loss = np.array(train_epoch_losses).mean()
    
    for ix, batch in enumerate(iter(trn_dl)):
        x, y = batch
        is_correct = accuracy(x, y, model)
        train_epoch_accuracies.extend(is_correct)
    train_epoch_accuracy = np.mean(train_epoch_accuracies)
    
    for ix, batch in enumerate(iter(val_dl)):
        x, y = batch
        val_is_correct = accuracy(x, y, model)
        val_epoch_accuracies.extend(val_is_correct)
    val_epoch_accuracy = np.mean(val_epoch_accuracies)
    
    train_losses.append(train_epoch_loss)
    train_accuracies.append(train_epoch_accuracy)
    val_accuracies.append(val_epoch_accuracy)

In [ ]:
epochs = np.arange(5)+1
import matplotlib.ticker as mtick


plt.plot(epochs, train_accuracies, 'bo', label="Training accuracy")
plt.plot(epochs, val_accuracies, 'r', label='Validation accuracy')
plt.gca().xaxis.set_major_locator(mtick.MultipleLocator(1))
plt.title("Training and Validation accuracy  with 4k data points used during training")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.gca().set_yticklabels(['{:.0f}%'.format(x*100) for x in plt.gca().get_yticks()])
plt.legend()
plt.grid('off')
plt.show()